# Trying Out the Hot v Cold Classifier

## Context
In this notebook, we will test the model we created in the previous build log. We'll do this testing by running the exported model on some data it's never seen (we have some spare data to do this with from the original experiments done in build log 1).

I'd like to have this model run on a website, so this notebook will double as a reference implementation of the inference pipeline to use on a web server.

## Dependencies

* Fast.ai `conda install -c fastai fastai`
    * Important note: fastai requires Python 3.6.*
* pytorch ([installation instructions](https://pytorch.org/))
* LibROSA `conda install librosa`

## Notebook Setup

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.vision import *
from fastai.metrics import error_rate

In [4]:
print("cuda available") if torch.cuda.is_available() else print("cuda not available")

print(torch.cuda.get_device_name(0))
print('Memory Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Memory Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

cuda available
GeForce GTX 970
Memory Allocated: 0.0 GB
Memory Cached:    0.0 GB


In [27]:
modelPath = Path('data-v2/spectrograms/'); print(modelPath)
audioPath  = Path('data/audio'); print(audioPath)
tempDir = Path(tempfile.mkdtemp()); print(tempDir)

data-v2\spectrograms
data\audio
C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4


## Generate Spectrograms for the data

In [28]:
# Constants
min_frequency=256
max_frequency = 16384
samples_between_frames = 256
fft_window_size = 2048

# Generate and save the spectrograms
import os
import librosa
import librosa.display
import matplotlib.pyplot

def generate_and_save_libROSA_spectrogram(audioFilePath):
    print('Generating spectrogram for: ' + str(audioFilePath))

    # Load the audio, generate a spectrogram in DB (to match the logarithmic nature of human hearing)
    audio_data, sample_rate = librosa.load(audioFilePath, sr=None)
    spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=128, fmax=max_frequency,
                                             hop_length=samples_between_frames, n_fft=fft_window_size)
    amplitude_in_db = librosa.power_to_db(spectrogram, ref=np.max)

    # Generate the spectrogram with min and max frequency bounds applied
    librosa.display.specshow(amplitude_in_db, x_axis=None, y_axis=None, sr=sample_rate, fmin=min_frequency, fmax=max_frequency)
    plt.tight_layout()
    
    # Save the generated spectrogram
    spectrogramSavePath = Path(tempDir/(os.path.splitext(os.path.basename(audioFilePath))[0] + '.png'))
    print('Saving spectrogram as: ' + str(spectrogramSavePath))
    plt.savefig(spectrogramSavePath)
    plt.close()


for fileName in os.listdir(audioPath):
    if re.match(r"\d+f-[a-zA-Z]+-\d+.m4a", fileName):
        fullName = audioPath/fileName
        generate_and_save_libROSA_spectrogram(fullName)

Generating spectrogram for: data\audio\175f-ceramicMug-1.m4a
Saving spectrogram as: C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4\175f-ceramicMug-1.png
Generating spectrogram for: data\audio\175f-glass-1.m4a
Saving spectrogram as: C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4\175f-glass-1.png
Generating spectrogram for: data\audio\175f-glassMug-1.m4a
Saving spectrogram as: C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4\175f-glassMug-1.png
Generating spectrogram for: data\audio\175f-smallThermos-1.m4a
Saving spectrogram as: C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4\175f-smallThermos-1.png
Generating spectrogram for: data\audio\195f-ceramicMug-1.m4a
Saving spectrogram as: C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4\195f-ceramicMug-1.png
Generating spectrogram for: data\audio\195f-ceramicMug-2.m4a
Saving spectrogram as: C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4\195f-ceramicMug-2.png
Generating spectrogram for: data\audio\195f-glassMug-1.m4a
Saving spectrogram as: C:\Users\cjjea\A

In [29]:
defaults.device = torch.device('cpu')
learn = load_learner(modelPath)

In [30]:
learn.model # Verify the model loaded

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [31]:
print(tempDir)
os.listdir(tempDir)

C:\Users\cjjea\AppData\Local\Temp\tmph8ws3hx4


['175f-ceramicMug-1.png',
 '175f-glass-1.png',
 '175f-glassMug-1.png',
 '175f-smallThermos-1.png',
 '195f-ceramicMug-1.png',
 '195f-ceramicMug-2.png',
 '195f-glassMug-1.png',
 '195f-smallThermos-1.png',
 '195f-smallThermos-2.png',
 '195f-smallThermos-3.png',
 '212f-ceramicMug-1.png',
 '212f-ceramicMug-2.png',
 '212f-ceramicMug-3.png',
 '212f-glassMug-1.png',
 '212f-glassMug-2.png',
 '212f-smallThermos-1.png',
 '40f-ceramicMug-1.png',
 '40f-glass-1.png',
 '40f-glassMug-1.png',
 '40f-smallThermos-1.png',
 '45f-ceramicMug-1.png',
 '45f-glass-1.png',
 '45f-glassMug-1.png',
 '45f-smallThermos-1.png',
 '73f-ceramicMug-1.png',
 '73f-ceramicMug-2.png',
 '73f-glass-1.png',
 '73f-glass-2.png',
 '73f-glassMug-1.png',
 '73f-glassMug-2.png',
 '73f-smallThermos-1.png',
 '73f-smallThermos-2.png']

In [38]:
for fileName in os.listdir(tempDir):
    print(fileName)
    pattern = re.compile(r'(\d+)f-[a-zA-Z]+-\d+.png')
    temp = pattern.search(str(fileName)).group(1)
    classification = 'hot' if int(temp) > 100 else 'cold'
    image = open_image(tempDir/fileName)
    pred_classification,pred_idx,outputs = learn.predict(image)
    print("Predicted: ")
    print(pred_classification)
    print("Confidence: ")
    print(outputs)
    print("actual: " + classification)
    print("\n-----\n")

175f-ceramicMug-1.png
Predicted: 
cold
Confidence: 
tensor([0.9837, 0.0163])
actual: hot

-----

175f-glass-1.png
Predicted: 
cold
Confidence: 
tensor([0.7226, 0.2774])
actual: hot

-----

175f-glassMug-1.png
Predicted: 
hot
Confidence: 
tensor([0.0011, 0.9989])
actual: hot

-----

175f-smallThermos-1.png
Predicted: 
hot
Confidence: 
tensor([8.0254e-04, 9.9920e-01])
actual: hot

-----

195f-ceramicMug-1.png
Predicted: 
hot
Confidence: 
tensor([3.9365e-08, 1.0000e+00])
actual: hot

-----

195f-ceramicMug-2.png
Predicted: 
hot
Confidence: 
tensor([3.8149e-07, 1.0000e+00])
actual: hot

-----

195f-glassMug-1.png
Predicted: 
hot
Confidence: 
tensor([5.8788e-09, 1.0000e+00])
actual: hot

-----

195f-smallThermos-1.png
Predicted: 
hot
Confidence: 
tensor([1.3353e-05, 9.9999e-01])
actual: hot

-----

195f-smallThermos-2.png
Predicted: 
hot
Confidence: 
tensor([9.0446e-07, 1.0000e+00])
actual: hot

-----

195f-smallThermos-3.png
Predicted: 
hot
Confidence: 
tensor([0.0143, 0.9857])
actual: hot

## Not too bad!
It looks like the model tends to err toward marking lower-temperature hot examples as cold, with only the first two predictions being incorrect (2 175 degree hot examples marked as cold). The errors were close to the boundary between hot and cold in the training data, so it's an understandable mis-classification.

There were 32 examples in this test, and only two were incorrectly classified, yielding a 6.25% error rate. Not too shabby.

This model could be further improved, but this is just a fun side-project, so I think it's ready to deploy.